In [1]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from baggingPU import BaggingClassifierPU

In [2]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

In [3]:
n_input = 14
n_sample = int(1e5)


df_raw = pd.read_csv('../data/3x+1.csv')

df_X = df_raw.iloc[:,:-1]
df_y = df_raw.iloc[:,-1]


print(df_X)
print(df_y)

       p1  p2  p3  p4    p5
0       1   0   0   0     0
1       0   0   1   0     0
2       0   1   0   0     0
3       0   0   0   1     0
4       0   0   0   1     1
...    ..  ..  ..  ..   ...
19995   1   1   1   1  5390
19996   1   0   1   1  3007
19997   1   0   1   0  1186
19998   1   1   1   1  3334
19999   1   0   0   1  7254

[20000 rows x 5 columns]
0        1
1        1
2        1
3        1
4        1
        ..
19995    0
19996    0
19997    0
19998    0
19999    0
Name: label, Length: 20000, dtype: int64


In [4]:
print(len(np.unique(np.array(df_X),axis = 0)))

print(bin(7499).replace('0b',''))

20000
1110101001011


In [5]:
def transformer(x):
    x_2 = bin(x).replace('0b','')
    x_raw = list(str(x_2))
    x = [0] * (n_input - len(x_raw)) + x_raw
    return pd.Series(x)

In [6]:
a = df_X.iloc[-1,-1]
b = transformer(a).values.astype(np.int)
print(b)

[0 1 1 1 0 0 0 1 0 1 0 1 1 0]


In [7]:
colums = []
col_df_raw = df_X.columns.values.tolist()
i = 4
for j in range(n_input):
    str1 = col_df_raw[i] + '-' + str(j)
    colums.append(str1)
print(colums)

['p5-0', 'p5-1', 'p5-2', 'p5-3', 'p5-4', 'p5-5', 'p5-6', 'p5-7', 'p5-8', 'p5-9', 'p5-10', 'p5-11', 'p5-12', 'p5-13']


In [8]:
df_new = pd.DataFrame(columns = colums)
print(df_new)

Empty DataFrame
Columns: [p5-0, p5-1, p5-2, p5-3, p5-4, p5-5, p5-6, p5-7, p5-8, p5-9, p5-10, p5-11, p5-12, p5-13]
Index: []


In [9]:
for i in range(20000):
    a = df_X.iloc[i,4]
    b = transformer(a).values.astype(np.int)
    se = pd.Series(b, colums)
    df_new.loc[i] = se
print(df_new)

      p5-0 p5-1 p5-2 p5-3 p5-4 p5-5 p5-6 p5-7 p5-8 p5-9 p5-10 p5-11 p5-12  \
0        0    0    0    0    0    0    0    0    0    0     0     0     0   
1        0    0    0    0    0    0    0    0    0    0     0     0     0   
2        0    0    0    0    0    0    0    0    0    0     0     0     0   
3        0    0    0    0    0    0    0    0    0    0     0     0     0   
4        0    0    0    0    0    0    0    0    0    0     0     0     0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
19995    0    1    0    1    0    1    0    0    0    0     1     1     1   
19996    0    0    1    0    1    1    1    0    1    1     1     1     1   
19997    0    0    0    1    0    0    1    0    1    0     0     0     1   
19998    0    0    1    1    0    1    0    0    0    0     0     1     1   
19999    0    1    1    1    0    0    0    1    0    1     0     1     1   

      p5-13  
0         0  
1         0  
2         0  
3         0  
4    

In [10]:
df_final = pd.concat([df_X.iloc[:,:-1],df_new],axis=1)
print(df_final)

       p1  p2  p3  p4 p5-0 p5-1 p5-2 p5-3 p5-4 p5-5 p5-6 p5-7 p5-8 p5-9 p5-10  \
0       1   0   0   0    0    0    0    0    0    0    0    0    0    0     0   
1       0   0   1   0    0    0    0    0    0    0    0    0    0    0     0   
2       0   1   0   0    0    0    0    0    0    0    0    0    0    0     0   
3       0   0   0   1    0    0    0    0    0    0    0    0    0    0     0   
4       0   0   0   1    0    0    0    0    0    0    0    0    0    0     0   
...    ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
19995   1   1   1   1    0    1    0    1    0    1    0    0    0    0     1   
19996   1   0   1   1    0    0    1    0    1    1    1    0    1    1     1   
19997   1   0   1   0    0    0    0    1    0    0    1    0    1    0     0   
19998   1   1   1   1    0    0    1    1    0    1    0    0    0    0     0   
19999   1   0   0   1    0    1    1    1    0    0    0    1    0    1     0   

      p5-11 p5-12 p5-13  
0

In [11]:
from sklearn.decomposition import PCA
dffinal = np.array(df_final)
print(dffinal)

[[1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 1 ... 0 1 0]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 0]]


In [12]:
pca = PCA(n_components=10)

dfraw4D = pca.fit_transform(dffinal[:,:]) 
print(dffinal)
print(dfraw4D)

[[1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 1 ... 0 1 0]
 [1 1 1 ... 1 1 0]
 [1 0 0 ... 1 1 0]]
[[ 0.00413557  0.1449494  -0.01446507 ... -0.2398823  -0.35288369
   0.12676176]
 [-0.70921627  0.04156085 -0.11079623 ... -0.31961482 -0.37830568
   0.14534373]
 [ 0.00526533  0.14179301 -0.00810289 ... -0.23950759 -0.35055533
   0.12560601]
 ...
 [-0.70064728  0.90545759 -0.24614229 ... -0.74328322 -0.5787547
  -0.18513219]
 [ 0.00538038  1.01964044  0.56022694 ...  0.27041423  0.31304559
  -0.61735039]
 [ 0.70861097  0.27696915  0.00836457 ... -0.34717201  0.45242314
  -0.75720712]]


In [13]:
print(pca.explained_variance_ratio_)

[0.1057102  0.06783682 0.06776781 0.06723643 0.06491253 0.0632845
 0.06247499 0.06211123 0.06196076 0.06180182]


In [14]:
df_aa = pd.DataFrame(dfraw4D)
print(df_aa)

              0         1         2         3         4         5         6  \
0      0.004136  0.144949 -0.014465  0.037175  0.682697  0.925760  0.547266   
1     -0.709216  0.041561 -0.110796 -0.385104  0.542553  0.919952  0.537883   
2      0.005265  0.141793 -0.008103  0.026870  0.688312  0.926434  0.545253   
3      0.704815  0.042959 -0.116697 -0.398849  0.533592  0.920207  0.529785   
4      0.704402 -0.013491 -0.149675 -0.412516  0.545991  0.562369  0.601882   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.006682  0.325161 -0.589376  1.073248  0.588939  0.975439 -0.218809   
19996 -0.003434 -0.007519  0.976972  0.663474 -0.681273 -0.490172 -0.852711   
19997 -0.700647  0.905458 -0.246142  0.304370  0.726010 -0.012484 -0.058517   
19998  0.005380  1.019640  0.560227  0.802950  0.576903  1.132988 -0.241154   
19999  0.708611  0.276969  0.008365  0.006728  0.827419  0.243331 -0.579848   

              7         8         9  
0     -0.2398

In [15]:
#use PCA

x_data = df_aa.iloc[:,:].copy()
y_data = df_y.iloc[:].copy()

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [15]:
#do not use PCA
x_data = df_final.iloc[:,:].copy()
y_data = df_y.iloc[:].copy()

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [16]:
x_train_input.head(10)

,p1,p2,p3,p4,p5-0,p5-1,p5-2,p5-3,p5-4,p5-5,p5-6,p5-7,p5-8,p5-9,p5-10,p5-11,p5-12,p5-13,label
8118,0,0,0,1,0,1,0,1,1,1,1,1,0,0,0,1,1,1,1
10171,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
9399,0,0,1,0,0,1,1,0,1,1,1,0,0,0,1,0,0,0,1
12669,0,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,0,0
18809,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
13869,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,0
3320,0,0,0,1,0,0,1,0,0,1,1,0,1,1,1,0,0,0,1
14689,0,0,1,0,0,1,1,0,1,1,0,1,1,1,1,0,1,0,0
13087,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,1,1,0
15992,0,1,0,1,0,1,0,1,1,1,0,1,0,1,1,1,1,1,0


In [17]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 3000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

In [18]:
pd.Series(y).value_counts()

0    10923
1     5077
Name: label, dtype: int64

In [19]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_raw.label), len(df_raw.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 18 features
- 10000 positive out of 20000 total before hiding labels
- 5077 positive out of 16000 total after hiding labels


# Trainning directly

In [20]:
print(X)
print(y)

       p1  p2  p3  p4 p5-0 p5-1 p5-2 p5-3 p5-4 p5-5 p5-6 p5-7 p5-8 p5-9 p5-10  \
8118    0   0   0   1    0    1    0    1    1    1    1    1    0    0     0   
10171   0   0   1   0    0    0    0    0    0    1    0    0    0    0     0   
9399    0   0   1   0    0    1    1    0    1    1    1    0    0    0     1   
12669   0   0   1   0    0    0    1    1    1    1    1    0    1    0     0   
18809   1   1   1   1    0    1    0    0    0    0    0    1    0    0     0   
...    ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
13927   0   0   1   0    0    1    0    1    1    1    0    0    0    0     0   
919     0   0   1   0    0    0    0    0    1    0    1    0    1    1     0   
5699    0   0   1   0    0    1    0    0    0    0    1    0    1    1     0   
10742   0   0   0   1    0    0    0    1    0    0    0    1    0    1     1   
16921   1   1   1   1    0    0    1    0    1    1    0    0    1    1     1   

      p5-11 p5-12 p5-13  
8

In [21]:
print('Training MLP model ...')

from sklearn.neural_network import MLPClassifier

model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(30,30), random_state=1,max_iter=200)

model.fit(X, y)

print('Done')

Training MLP model ...
Done


In [22]:
model.predict(X)

array([1, 0, 1, ..., 1, 0, 0])

In [23]:
print('---- {} ----'.format('MLP model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- MLP model ----
                        pred_negative        pred_positive 
           true_negative       7528.0        395.0 
           true_positive       1486.0       6591.0 
None

Precision:  0.9434583452619525
Recall:  0.8160207998019067
Accuracy:  0.8824375
f1_score:  0.8751244771957778


# Training by bagging

In [24]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(30,30), random_state=1,max_iter=300)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import GaussianNB
model5 = GaussianNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [25]:
model_list = [model1, model2, model3, model4, model5, model6, model7]
f1 = np.zeros([len(model_list)],dtype=np.float32)

In [34]:
for i,j in zip(model_list,range(len(model_list))):
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(i,
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('---- {} ----'.format(i))
    print('Time:', pu_end - pu_start)
    print(f1_score(y_orig, pre))
    f1[j] = f1_score(y_orig, pre)

print(f1)

---- LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) ----
Time: 1.2592580090276897
0.8044225310025399
---- DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') ----
Time: 0.5238404399715364
0.7333672560742908
---- MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon

In [35]:
best_model = BaggingClassifierPU(model_list[np.argmax(f1)],
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
best_model.fit(X,y)
pre = best_model.predict(X)
print(model)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [36]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, pre))
print('Recall: ', recall_score(y_orig, pre))
print('Accuracy: ', accuracy_score(y_orig, pre))
print('f1_score: ', f1_score(y_orig, pre))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7887.0         36.0 
           true_positive          7.0       8070.0 
None

Precision:  0.9955588452997779
Recall:  0.999133341587223
Accuracy:  0.9973125
f1_score:  0.9973428906877587


In [37]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(X.loc[FN_index])
print("False Positive:")
print(X.loc[FT_index])

False Negtive:
      p1  p2  p3  p4 p5-0 p5-1 p5-2 p5-3 p5-4 p5-5 p5-6 p5-7 p5-8 p5-9 p5-10  \
9399   0   0   1   0    0    1    1    0    1    1    1    0    0    0     1   
4446   0   0   0   1    0    0    1    1    0    1    0    0    0    0     0   
9445   0   0   1   0    0    1    1    0    1    1    1    0    1    0     1   
9620   0   0   0   1    0    1    1    1    0    0    0    0    1    0     1   
6806   0   0   0   1    0    1    0    0    1    1    1    1    1    0     1   
...   ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
2758   0   0   0   1    0    0    1    0    0    0    0    0    0    1     0   
349    0   0   1   0    0    0    0    0    0    1    0    0    0    0     0   
1439   0   0   1   0    0    0    0    1    0    0    0    0    1    1     0   
4859   0   0   1   0    0    0    1    1    1    0    0    0    1    1     1   
6118   0   0   0   1    0    1    0    0    0    1    1    1    1    0     1   

     p5-11 p5-12 p5-13  

In [38]:
pre = best_model.predict(x_test)
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, pre), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, pre))
print('Recall: ', recall_score(y_test, pre))
print('Accuracy: ', accuracy_score(y_test, pre))
print('f1_score: ', f1_score(y_test, pre))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       2058.0         19.0 
           true_positive          1.0       1922.0 
None

Precision:  0.9902112313240597
Recall:  0.999479979199168
Accuracy:  0.995
f1_score:  0.994824016563147


In [39]:
#print wrong predictions
y_test_pre = model.predict(x_test)
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(x_test.loc[FN_test_index])
print("False Positive:")
print(x_test.loc[FT_test_index])

False Negtive:
      p1  p2  p3  p4 p5-0 p5-1 p5-2 p5-3 p5-4 p5-5 p5-6 p5-7 p5-8 p5-9 p5-10  \
5093   0   0   1   0    0    0    1    1    1    0    1    1    1    0     1   
2853   0   0   1   0    0    0    1    0    0    0    0    1    0    1     1   
4752   0   0   0   1    0    0    1    1    0    1    1    1    1    0     1   
4536   0   0   0   1    0    0    1    1    0    1    0    1    0    0     1   
4000   0   0   0   1    0    0    1    0    1    1    1    0    1    1     0   
...   ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
9223   0   0   1   0    0    1    1    0    1    1    0    0    0    0     0   
2293   0   0   1   0    0    0    0    1    1    0    1    0    1    1     0   
4990   0   0   0   1    0    0    1    1    1    0    1    0    0    1     1   
4548   0   0   0   1    0    0    1    1    0    1    0    1    0    1     0   
997    0   0   1   0    0    0    0    0    1    0    1    1    1    0     1   

     p5-11 p5-12 p5-13  